In [1]:
# Install necessary packages
!pip install -q openai langchain langchain_community

# Import required libraries
from openai import OpenAI
from google.colab import userdata
from langchain_community.llms import OpenAI as LangChainOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import List
import json
import os

# Documentation:
# 1. This section installs and imports essential libraries for interacting with OpenAI and LangChain.
# 2. Ensure your environment supports internet access to install packages.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [2]:
# Set up API keys for OpenAI
open_ai_api_key = userdata.get('OPENAI_KEY')  # Replace with your OpenAI API key if not using Colab's userdata
hugging_face_token = userdata.get('HF_TOKEN')  # For Hugging Face, optional
os.environ["OPENAI_API_KEY"] = open_ai_api_key



# Define Fields and Prompts

In [3]:
# Define diverse fields and prompts for different scenarios
prompts_new = {
    'E-commerce': [
        {
            'prompt': '''### Instruction:
Write a detailed product description for a luxury smartwatch highlighting its features, materials, and unique selling points.

### Output format:
- **Product Name**:
- **Features**:
- **Materials**:
- **Unique Selling Points**:
''',
            'test_case': {}
        },
        {
            'prompt': '''### Instruction:
Draft a promotional email for a flash sale event for an online fashion store, including the sale duration, featured categories, and a call to action.

### Output format:
- **Subject Line**:
- **Introduction**:
- **Featured Categories**:
- **Call to Action**:
''',
            'test_case': {}
        },
        {
            'prompt': '''### Instruction:
Develop a FAQ section for an online electronics store.

### Output format:
- **Question 1**:
  - **Answer**:
- **Question 2**:
  - **Answer**:
(Include at least 5 questions and answers)
''',
            'test_case': {}
        }
    ],
    'Education': [
        {
            'prompt': '''### Instruction:
Create a course syllabus for a 10-week programming class for beginners. Include weekly topics, learning objectives, and required materials.

### Output format:
- **Week 1**:
  - Topics:
  - Objectives:
  - Materials:
...
(Repeat for all 10 weeks)
''',
            'test_case': {}
        },
        {
            'prompt': '''### Instruction:
Design a 5-question pop quiz on basic algebra for middle school students.

### Output format:
Provide each question and four options, marking the correct answer.

- **Question 1**:
  - A.
  - B.
  - C.
  - D.
- **Answer**:
(Repeat for all questions)
''',
            'test_case': {}
        },
        {
            'prompt': '''### Instruction:
Write a motivational message for students preparing for final exams.

### Output format:
- **Introduction**:
- **Encouragement**:
- **Tips for Success**:
- **Closing**:
''',
            'test_case': {}
        }
    ]
}

# Documentation:
# 1. This dictionary contains new fields and prompts for different use cases (e.g., E-commerce, Education).
# 2. Prompts are structured to include specific output formats for consistency and ease of validation.


Testing Function

In [4]:
# Function to test individual prompts
def test_prompt_new(prompt_text, model="gpt-3.5-turbo", max_tokens=300):
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt_text}],
        max_tokens=max_tokens,
        temperature=0  # Set for deterministic results
    )
    return response.choices[0].message.content.strip()

# Documentation:
# - This function sends a prompt to the OpenAI API and retrieves a formatted response.
# - Adjust `max_tokens` as needed based on the complexity of your prompts.


Testing Specific Field

In [5]:
# Function to retrieve and test prompts for a specific field
def get_field_outputs_new(field_name):
    if field_name in prompts_new:
        outputs = []
        for item in prompts_new[field_name]:
            output = test_prompt_new(item['prompt'])
            outputs.append({'prompt': item['prompt'], 'output': output})
        return outputs
    else:
        print(f"Field '{field_name}' not found.")
        return None

# Documentation:
# - This function loops through all prompts in a given field and collects their outputs.
# - Use this function to test specific scenarios or validate prompt behavior.


Display Results

In [7]:
# Define Prompt Templates for Various Categories
prompts = {
    "E-commerce": [
        {
            "prompt": """### Instruction:\nCreate a product description for a new fitness tracker, including its features, benefits, and target audience.\n\n### Output format:\n- **Product Name**:\n- **Features**:\n- **Benefits**:\n- **Target Audience**:""",
            "expected_output": {
                "Product Name": "FitPlus Tracker",
                "Features": ["Step counter", "Heart rate monitor", "Sleep tracker", "Water-resistant"],
                "Benefits": ["Helps monitor fitness goals", "Tracks heart health", "Improves sleep quality"],
                "Target Audience": ["Fitness enthusiasts", "Health-conscious individuals"]
            },
        },
        {
            "prompt": """### Instruction:\nWrite an email to customers announcing a seasonal sale on electronics, offering a 20% discount on all products.\n\n### Output format:\n- **Subject**:\n- **Greeting**:\n- **Announcement Details**:\n- **Call to Action**:""",
            "expected_output": {
                "Subject": "Big Savings on Electronics - 20% Off!",
                "Greeting": "Dear Valued Customer",
                "Announcement Details": "We are excited to announce our seasonal sale, offering 20% off on all electronics.",
                "Call to Action": "Shop now and enjoy the discount!"
            },
        }
    ]
}

# Function to Test a Single Prompt
def test_single_prompt(prompt):
    """
    Tests a single prompt using OpenAI's GPT API.
    Args:
        prompt (str): The input prompt to test.
    Returns:
        str: The response generated by the GPT model.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300,
            temperature=0
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        return f"Error: {e}"

# Documentation:
# - The `test_single_prompt` function sends a single prompt to the OpenAI API.
# - Handles errors gracefully, returning error details if encountered.


In [10]:
# Install and import required libraries
!pip install -q openai langchain

import openai
import os

# Documentation:
# - Ensure you have set your OpenAI API key using the environment variable `OPENAI_API_KEY`.
# - Upgrade OpenAI if needed using `!pip install --upgrade openai`.
# - This setup is compatible with the latest OpenAI API version.

# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"  # Replace with your API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to test prompts using the OpenAI ChatCompletion API
def test_prompt(prompt, model="gpt-3.5-turbo", max_tokens=300):
    """
    Sends a prompt to the OpenAI ChatCompletion API and retrieves the response.

    Args:
        prompt (str): The input prompt for the model.
        model (str): The model to use, default is "gpt-3.5-turbo".
        max_tokens (int): Maximum number of tokens in the response.

    Returns:
        str: The generated response from the model or an error message.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        return f"Error: {e}"

# Prompts for Testing
prompts = {
    "E-commerce": [
        {
            "prompt": """### Instruction:\nCreate a product description for a new fitness tracker, including its features, benefits, and target audience.\n\n### Output format:\n- **Product Name**:\n- **Features**:\n- **Benefits**:\n- **Target Audience**:""",
            "expected_output": {
                "Product Name": "FitPlus Tracker",
                "Features": ["Step counter", "Heart rate monitor", "Sleep tracker", "Water-resistant"],
                "Benefits": ["Helps monitor fitness goals", "Tracks heart health", "Improves sleep quality"],
                "Target Audience": ["Fitness enthusiasts", "Health-conscious individuals"]
            },
        },
        {
            "prompt": """### Instruction:\nWrite an email to customers announcing a seasonal sale on electronics, offering a 20% discount on all products.\n\n### Output format:\n- **Subject**:\n- **Greeting**:\n- **Announcement Details**:\n- **Call to Action**:""",
            "expected_output": {
                "Subject": "Big Savings on Electronics - 20% Off!",
                "Greeting": "Dear Valued Customer",
                "Announcement Details": "We are excited to announce our seasonal sale, offering 20% off on all electronics.",
                "Call to Action": "Shop now and enjoy the discount!"
            },
        }
    ]
}

# Function to Pretty-Print Prompt Outputs
def pretty_print_outputs_new(category):
    """
    Pretty-prints the prompts and their generated outputs for a specific category.

    Args:
        category (str): The category of prompts to test.
    """
    if category in prompts:
        print(f"--- Testing Prompts for Category: {category} ---\n")
        for idx, item in enumerate(prompts[category]):
            print(f"Prompt {idx + 1}:\n")
            print(item["prompt"])
            print("\nExpected Output:\n", item["expected_output"])
            print("\nGenerated Output:\n", test_prompt(item["prompt"]))
            print("\n" + "-" * 50 + "\n")
    else:
        print(f"Category '{category}' not found.")

# Example Execution
pretty_print_outputs_new("E-commerce")


--- Testing Prompts for Category: E-commerce ---

Prompt 1:

### Instruction:
Create a product description for a new fitness tracker, including its features, benefits, and target audience.

### Output format:
- **Product Name**:
- **Features**:
- **Benefits**:
- **Target Audience**:

Expected Output:
 {'Product Name': 'FitPlus Tracker', 'Features': ['Step counter', 'Heart rate monitor', 'Sleep tracker', 'Water-resistant'], 'Benefits': ['Helps monitor fitness goals', 'Tracks heart health', 'Improves sleep quality'], 'Target Audience': ['Fitness enthusiasts', 'Health-conscious individuals']}

Generated Output:
 Error: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed m